In [87]:
import sqlite3
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [88]:
boardgames = pd.read_csv("boardgames.csv")
boardgames

,title,rating,type,min_age,player_count,playing_time,weight,description
0,Brass: Birmingham,8.6,Strategy,14.0,2–4–,60–120–,3.87,Brass: Birminghamis an economic strategy game ...
1,Pandemic Legacy: Season 1,8.5,Strategy,13.0,2–4–,60,2.83,Pandemic Legacyis a co-operative campaign game...
2,Ark Nova,8.5,Strategy,14.0,1–4–,90–150–,3.78,"InArk Nova, you will plan and design a modern,..."
3,Gloomhaven,8.6,Strategy,14.0,1–4–,60–120–,3.91,This is a game with a persistent and changing ...
4,Twilight Imperium: Fourth Edition,8.6,Strategy,14.0,3–6–,240–480–,4.33,Twilight Imperium (Fourth Edition)is a game of...
...,...,...,...,...,...,...,...,...
895,Disney Villainous: Perfectly Wretched,7.6,Family,10.0,2–3–,40–60–,2.31,"InDisney Villainous: Perfectly Wretched, each ..."
896,A War of Whispers,7.3,Strategy,14.0,2–4–,30–60–,2.57,A War of Whispersis a competitive board game f...
897,Warhammer: Invasion,7.2,Customizable,13.0,2,45,2.74,Warhammer: Invasion The Card Game is a two-pla...
898,878 Vikings: Invasions of England,7.5,Wargames,12.0,2–4–,60–120–,2.61,"The year is 878. For the past 75 years, Viking..."


In [89]:
# Remove trailing dash or em dash
boardgames['player_count'] = boardgames['player_count'].str.rstrip('-–')
boardgames['playing_time'] = boardgames['playing_time'].str.rstrip('-–')
boardgames

,title,rating,type,min_age,player_count,playing_time,weight,description
0,Brass: Birmingham,8.6,Strategy,14.0,2–4,60–120,3.87,Brass: Birminghamis an economic strategy game ...
1,Pandemic Legacy: Season 1,8.5,Strategy,13.0,2–4,60,2.83,Pandemic Legacyis a co-operative campaign game...
2,Ark Nova,8.5,Strategy,14.0,1–4,90–150,3.78,"InArk Nova, you will plan and design a modern,..."
3,Gloomhaven,8.6,Strategy,14.0,1–4,60–120,3.91,This is a game with a persistent and changing ...
4,Twilight Imperium: Fourth Edition,8.6,Strategy,14.0,3–6,240–480,4.33,Twilight Imperium (Fourth Edition)is a game of...
...,...,...,...,...,...,...,...,...
895,Disney Villainous: Perfectly Wretched,7.6,Family,10.0,2–3,40–60,2.31,"InDisney Villainous: Perfectly Wretched, each ..."
896,A War of Whispers,7.3,Strategy,14.0,2–4,30–60,2.57,A War of Whispersis a competitive board game f...
897,Warhammer: Invasion,7.2,Customizable,13.0,2,45,2.74,Warhammer: Invasion The Card Game is a two-pla...
898,878 Vikings: Invasions of England,7.5,Wargames,12.0,2–4,60–120,2.61,"The year is 878. For the past 75 years, Viking..."


In [90]:
# Split on dash (en-dash or em-dash), convert to int
split_counts_players = boardgames['player_count'].str.split('–', expand=True)
split_counts_time = boardgames['playing_time'].str.split('–', expand=True)

# If only one value (e.g. "2"), use it for both min and max
boardgames['max_players'] = split_counts_players[1].fillna(split_counts_players[0]).astype(float).astype('Int64')

boardgames['max_time'] = split_counts_time[1].fillna(split_counts_time[0]).astype(float).astype('Int64')

boardgames

,title,rating,type,min_age,player_count,playing_time,weight,description,max_players,max_time
0,Brass: Birmingham,8.6,Strategy,14.0,2–4,60–120,3.87,Brass: Birminghamis an economic strategy game ...,4,120
1,Pandemic Legacy: Season 1,8.5,Strategy,13.0,2–4,60,2.83,Pandemic Legacyis a co-operative campaign game...,4,60
2,Ark Nova,8.5,Strategy,14.0,1–4,90–150,3.78,"InArk Nova, you will plan and design a modern,...",4,150
3,Gloomhaven,8.6,Strategy,14.0,1–4,60–120,3.91,This is a game with a persistent and changing ...,4,120
4,Twilight Imperium: Fourth Edition,8.6,Strategy,14.0,3–6,240–480,4.33,Twilight Imperium (Fourth Edition)is a game of...,6,480
...,...,...,...,...,...,...,...,...,...,...
895,Disney Villainous: Perfectly Wretched,7.6,Family,10.0,2–3,40–60,2.31,"InDisney Villainous: Perfectly Wretched, each ...",3,60
896,A War of Whispers,7.3,Strategy,14.0,2–4,30–60,2.57,A War of Whispersis a competitive board game f...,4,60
897,Warhammer: Invasion,7.2,Customizable,13.0,2,45,2.74,Warhammer: Invasion The Card Game is a two-pla...,2,45
898,878 Vikings: Invasions of England,7.5,Wargames,12.0,2–4,60–120,2.61,"The year is 878. For the past 75 years, Viking...",4,120


In [91]:
boardgames['description'].iloc[0]

"Brass: Birminghamis an economic strategy game sequel to Martin Wallace's 2007 masterpiece,Brass.Brass: Birminghamtells the story of competing entrepreneurs in Birmingham during the industrial revolution between the years of 1770 and 1870.\nIt offers a very different story arc and experience from its predecessor. As in its predecessor, you must develop, build and establish your industries and network in an effort to exploit low or high market demands. The game is played over two halves: the canal era (years 1770-1830) and the rail era (years 1830-1870). To win the game, score the most VPs. VPs are counted at the end of each half for the canals, rails and established (flipped) industry tiles.\nEach round, players take turns according to the turn order track, receiving two actions to perform any of the following actions (found in the original game):\n1)Build- Pay required resources and place an industry tile.2)Network- Add a rail / canal link, expanding your network.3)Develop- Increase t

In [92]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_for_nlp(text):
    if pd.isna(text):
        return ""

    # 1. Lowercase
    text = text.lower()

    # 2. Fix stuck words and spacing
    text = re.sub(r'(?<=[.?!])(?=[^\s])', ' ', text)
    text = re.sub(r'(?<=[a-z])(?=[A-Z])', ' ', text)
    text = re.sub(r'(\d\))(?=\w)', r'\1 ', text)
    text = text.replace('\n', ' ')

    # 3. Remove punctuation and numbers
    text = re.sub(r'[^a-z\s]', ' ', text)

    # 4. Tokenize
    tokens = nltk.word_tokenize(text)

    # 5. Remove stopwords + lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    # 6. Rejoin
    return ' '.join(tokens)

In [93]:
boardgames['nlp_description'] = boardgames['description'].apply(clean_for_nlp)
boardgames = boardgames[['title', 'rating', 'type', 'min_age', 'max_players', 'max_time', 'weight', 'nlp_description']].dropna()

In [94]:
boardgames

,title,rating,type,min_age,max_players,max_time,weight,nlp_description
0,Brass: Birmingham,8.6,Strategy,14.0,4,120,3.87,brass birminghamis economic strategy game sequ...
1,Pandemic Legacy: Season 1,8.5,Strategy,13.0,4,60,2.83,pandemic legacyis co operative campaign game o...
2,Ark Nova,8.5,Strategy,14.0,4,150,3.78,inark nova plan design modern scientifically m...
3,Gloomhaven,8.6,Strategy,14.0,4,120,3.91,game persistent changing world ideally played ...
4,Twilight Imperium: Fourth Edition,8.6,Strategy,14.0,6,480,4.33,twilight imperium fourth edition game galactic...
...,...,...,...,...,...,...,...,...
895,Disney Villainous: Perfectly Wretched,7.6,Family,10.0,3,60,2.31,indisney villainous perfectly wretched player ...
896,A War of Whispers,7.3,Strategy,14.0,4,60,2.57,war whispersis competitive board game player f...
897,Warhammer: Invasion,7.2,Customizable,13.0,2,45,2.74,warhammer invasion card game two player card g...
898,878 Vikings: Invasions of England,7.5,Wargames,12.0,4,120,2.61,year past year viking raiding party norway den...


In [96]:
boardgames.to_csv("cleaned_boardgames.csv", index=False)